In [2]:
from random import randrange
from typing import List
from networkx import Graph

graphs: List[Graph] = []

graph_sizes = [4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 18, 20]
num_graph_per_size = 4

for size in graph_sizes:
    for i in range(num_graph_per_size):
        graph = Graph()
        num_edges = randrange(2, size*(size-2))  # rand in range of max number of possible edges
        edges = []

        for _ in range(num_edges):  # add random number of edges

            u = randrange(size-1)
            v = randrange(size-1)
            w = randrange(1, 4)

            if (u, v) not in [(u1, v1) for (u1, v1, _) in edges]:
                edges.append((u, v, w))

        for node in range(size):  # add an edge for any nodes not connected in the graph
            if node not in [u for (u, _, _) in edges] and node not in [v for (_, v, _) in edges]:
                u = node
                v = randrange(size-1)
                w = randrange(1, 4)

                edges.append((u, v, w))

        graph.add_weighted_edges_from(edges)
        graphs.append(graph)

len(graphs)

48

In [8]:
from copy import deepcopy
import json
from pathlib import Path
from bachelorthesis.graph_optimization import GraphOptimization

from bachelorthesis.solvers.qb_solv import solve as solve_qbs
from bachelorthesis.solvers.repeated_simulated_annealing import solve as solve_rsa
from bachelorthesis.solvers.simulated_annealing import solve as solve_sa
from bachelorthesis.solvers.tabu_search import solve as solve_tbs

with open(Path("graph_optimizations.json")) as file:
    optimization_parameters = json.load(file)

for graph in graphs:
    g_opt = GraphOptimization(graph=graph)
    n = graph.order()

    start_node = 0,
    terminal_node = n

    # positions question -> normally just n, but that doesn't make much sense for coloring style problems
    # then lower n make HTC impossible -> but if we're comparing relative solution quality

    for params in optimization_parameters:
        print(params)
        # allow positions in the json to take on a value of "n" for faster problem creation
        params_copy = deepcopy(params)
        p = params_copy.pop("positions")
        if p == "n":
            p = graph.order()

        # generate qubo
        qubo = g_opt.generate_qubo(
            positions=p,  # TODO: see above
            start_node=start_node,
            terminal_node=terminal_node,
            **params_copy
        )

        print(qubo)

        print(solve_qbs(qubo))



        # solve qubo and store result with all the relevant solvers


{'positions': 1, 'nodes_with_edges': -5, 'edges': 10}
[[ -5.  10.   0.   0.]
 [  0.  -5.  10.   0.]
 [  0.   0. -10.  10.]
 [  0.   0.   0.  -5.]]
(array([[1, 0, 1, 0]], dtype=int8), array([-15.]))
{'positions': 1, 'nodes_with_edges': -5, 'edges': 10}
[[-10.  10.  10.   0.]
 [  0.  -5.   0.   0.]
 [  0.   0. -10.  10.]
 [  0.   0.   0.  -5.]]
(array([[0, 1, 1, 0],
       [1, 0, 0, 1]], dtype=int8), array([-15., -15.]))
{'positions': 1, 'nodes_with_edges': -5, 'edges': 10}
[[ -5.  10.   0.   0.]
 [  0. -15.  10.  10.]
 [  0.   0.   0.   0.]
 [  0.   0.   0.  -5.]]
(array([[0, 1, 0, 0]], dtype=int8), array([-15.]))
{'positions': 1, 'nodes_with_edges': -5, 'edges': 10}
[[-15.  10.  10.  10.]
 [  0.  -5.   0.   0.]
 [  0.   0.  -5.   0.]
 [  0.   0.   0.  -5.]]
(array([[0, 1, 1, 1],
       [1, 0, 0, 0]], dtype=int8), array([-15., -15.]))
{'positions': 1, 'nodes_with_edges': -5, 'edges': 10}
[[  0.  10.   0.   0.   0.]
 [  0. -15.   0.  10.  10.]
 [  0.   0.  -5.  10.   0.]
 [  0.   0.   0.

In [ ]:
# solvers to use?
# nico used SA, RRSA, SAGA, TABU
# parameterization?